In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [4]:
import importlib
import GModReLU as GModReLUMod
importlib.reload(GModReLUMod)
from GModReLU import GModReLU, GModReLUFunction, LGRLinear,LGRConv2d

SyntaxError: invalid syntax (GModReLU.py, line 141)

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Define transform
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

full_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

from torch.utils.data import random_split

# Split into train and val
train_size = int(0.9 * len(full_train))
val_size = len(full_train) - train_size
train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch.nn as nn

class LGRCNN(nn.Module):
    def __init__(self, l=0.01, k=5.0):
        super().__init__()
        
        self.conv1 = LGRConv2d(3, 32, kernel_size=3, padding=1, l=l, k=k)
        self.conv2 = LGRConv2d(32, 64, kernel_size=3, padding=1, l=l, k=k)
        self.conv3 = LGRConv2d(64, 128, kernel_size=3, padding=1, l=l, k=k)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = LGRLinear(128 * 4 * 4, 128, l=l, k=k)
        self.fc2 = nn.Linear(128, 10)  # Final layer can stay standard (no modulation)

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [5]:
dtype = torch.float
device = torch.device("cpu")

In [9]:
l_val = 0.01
k_val = 10
model = LGRCNN(l = l_val, k = k_val)

In [12]:
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

In [20]:
from tqdm import tqdm
from collections import defaultdict, deque

In [22]:
epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    grad_norms = []
    weight_updates = []
    
    logs = defaultdict(list)
    running_loss = deque(maxlen=50)  # For smooth display
    running_acc = deque(maxlen=50)
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for batch_idx, (inputs, labels) in enumerate(pbar):
        inputs, labels = inputs.to(device), labels.to(device)
    
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
    
        # Gradients and updates
        total_norm = 0
        weight_update_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                total_norm += p.grad.data.norm(2).item() ** 2
                weight_update_norm += (lr * p.grad.data).norm(2).item() ** 2
    
        grad_norms.append(np.sqrt(total_norm))
        weight_updates.append(np.sqrt(weight_update_norm))
    
        optimizer.step()
    
        batch_size = inputs.size(0)
        total_loss += loss.item() * batch_size
        total += batch_size
    
        batch_acc = (outputs.argmax(1) == labels).float().mean().item()
        correct += (outputs.argmax(1) == labels).sum().item()
    
        # if batch_idx % log_every == 0:
        #     logs['batches'].append({
        #         'epoch': epoch,
        #         'batch_idx': batch_idx,
        #         'loss': loss.item(),
        #         'acc': batch_acc,
        #         'grad_norm': grad_norms[-1],
        #         'weight_update_norm': weight_updates[-1],
        #         'samples_seen': total
        #     })
    
        # Track running stats for display
        running_loss.append(loss.item())
        running_acc.append(batch_acc)
    
        if batch_idx % 10 == 0:
            pbar.set_postfix({
                'loss': f'{np.mean(running_loss):.4f}',
                'acc': f'{np.mean(running_acc) * 100:.2f}%',
                'grad_norm': f'{grad_norms[-1]:.2f}'
            })
    
    logs['train_loss'].append(total_loss / total)
    logs['train_acc'].append(correct / total)
    logs['train_grad_norm'].append(np.mean(grad_norms))
    logs['train_weight_update_norm'].append(np.mean(weight_updates))
    
    val_loss, val_acc = validate_model(model,val_loader,device)
    
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {logs['train_loss'][-1]:.4f} | "
          f"Train Acc: {logs['train_acc'][-1]*100:.2f}% | "
          f"Val Acc: {logs['val_acc'][-1]*100:.2f}%")

NameError: name 'validate_model' is not defined

In [34]:
optimizer.state_dict()['state'][0]['exp_avg_sq'].shape

torch.Size([32, 3, 3, 3])

In [43]:
optimizer.state_dict()['state'][1]['exp_avg_sq']

tensor([0.0156, 0.0022, 0.0042, 0.0110, 0.0039, 0.0025, 0.0014, 0.0112, 0.0022,
        0.0028, 0.0008, 0.0027, 0.0037, 0.0033, 0.0014, 0.0030, 0.0073, 0.0040,
        0.0021, 0.0040, 0.0156, 0.0012, 0.0142, 0.0096, 0.0014, 0.0066, 0.0032,
        0.0020, 0.0011, 0.0008, 0.0027, 0.0146])

In [48]:
lgr_linear = LGRLinear(128 * 4 * 4, 128, l=l_val, k=k_val)

In [79]:
optimizer.state_dict()['state'][1]['exp_avg_sq']

tensor([0.0156, 0.0022, 0.0042, 0.0110, 0.0039, 0.0025, 0.0014, 0.0112, 0.0022,
        0.0028, 0.0008, 0.0027, 0.0037, 0.0033, 0.0014, 0.0030, 0.0073, 0.0040,
        0.0021, 0.0040, 0.0156, 0.0012, 0.0142, 0.0096, 0.0014, 0.0066, 0.0032,
        0.0020, 0.0011, 0.0008, 0.0027, 0.0146])

In [62]:
model.state_dict()['conv1.bias'].shape

torch.Size([32, 3, 3, 3])

In [69]:
list(model.state_dict().keys())

['conv1.weight',
 'conv1.bias',
 'conv2.weight',
 'conv2.bias',
 'conv3.weight',
 'conv3.bias',
 'fc1.weight',
 'fc1.bias',
 'fc2.weight',
 'fc2.bias']

In [51]:
lgr_linear.state_dict()

OrderedDict([('weight',
              tensor([[-0.0175, -0.0126, -0.0215,  ..., -0.0125,  0.0165, -0.0206],
                      [ 0.0028,  0.0102, -0.0090,  ..., -0.0056,  0.0098, -0.0034],
                      [ 0.0016, -0.0030, -0.0073,  ...,  0.0173,  0.0062,  0.0142],
                      ...,
                      [-0.0081, -0.0192, -0.0060,  ..., -0.0066, -0.0157, -0.0179],
                      [ 0.0097, -0.0153, -0.0042,  ...,  0.0203, -0.0189, -0.0202],
                      [ 0.0160,  0.0105,  0.0142,  ..., -0.0122, -0.0215,  0.0133]])),
             ('bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 